In [4]:
# import SharedArray as sa
# ROBOT_TYPE_NAME = 'panda'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "success"]

In [2]:
RTYPE = "indy7"
DAT_DIR = '20210613-035257'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210613-035119'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 10
VISUALIZE = False
PLAY_RESULT = False
TIMEOUT_MOTION = 5
MAX_TIME = 100
SHOW_STATE = False


### test

In [3]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

### load results

In [4]:
res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
    assert len(res_dict_all[f_option][cname]) == int(fid)
    res_dict_all[f_option][cname].append([resdat[head] for head in HEADS])

In [6]:
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    print("="*30 + " {:<15} ".format(ekey) + "="*30)
    print("{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format("Checker", "Planning time", "Motion count", "MP trials", "Failed MP", "Failures", "Total"))
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])[:10,:]
            if len(vv)>0:
                succ_vec = vv[:,-1]
                fail_vec = np.logical_not(succ_vec)
                idc_succ = np.where(succ_vec)[0]
                print(
                    "{:<15}: {:<15} {:<15} {:<15} {:<15} {:<15} {:<15}".format(cname,
                                    *(list(np.round(np.mean(vv[idc_succ, :-1], axis=0), 2)) \
                                      +[np.sum(fail_vec), len(vv)])))
    print(" ")

============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 7.2             5.8             3.4             0.0             0               10             
Tool           : 7.83            6.5             4.5             0.3             0               10             
ToolReach      : 3.55            5.8             4.2             0.3             0               10             
Full           : 1.94            5.4             3.2             0.0             0               10             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 23.27           8.11            4.56            0.0             1               10             
Tool           : 11.77           5.5             6.

## With IK, average for step1~3 all success 

In [59]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "stowing")
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, "stowing-result")
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_tot", "MP_count", "failed_MPs", "IK_tot", "IK_count", "failed_IKs", "success"]
DISPS = ["Time", "Actions", "MP_tot", "MPs", "MP fails", "IK_tot", "IKs", "IK fails", "Failures", "Total"]
H2D = {}
D2H = {}
for head, disp in zip(HEADS, DISPS):
    H2D[head] = disp
    D2H[disp] = head

# RTYPE = "indy7gripper"
# DAT_DIR = '20210702-152658'
# FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

RTYPE = "panda"
DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = True
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

res_dict_all = defaultdict(lambda: defaultdict(list))
resfile_list = sorted(os.listdir(os.path.join(RESULTSET_PATH)))
for resfile in resfile_list:
    args = resfile[:-4].split("_")[1:]
    f_option = "_".join(args[:-2])
    fid, cname = args[-2:]
    resdat = load_pickle(os.path.join(RESULTSET_PATH, resfile))
    res_dict_all[f_option][cname].append([resdat[head] if head in resdat else None for head in HEADS])
    
exp_list = ['obj_1', 'obj_3', 'obj_3_pole']
for ekey in exp_list:
    if ekey not in res_dict_all:
        continue
    res_dict = res_dict_all[ekey]
    
    succ_vec_list = []
    len_list = []
    None_cols = []
    for cname in CNAME_LIST:
        if cname in res_dict:
            vv = np.array(res_dict[cname])
            if len(vv)>0:
                None_cols.append(np.any(np.equal(vv, None), axis=0))
                succ_vec = vv[:,-1]
                succ_vec_list.append(succ_vec)
                len_list.append(len(succ_vec))
                if cname == "None":
                    fail_vec = np.logical_not(succ_vec)
                    idc_fail_none = np.where(fail_vec)[0]
    print("="*30 + " {:<10} ".format(ekey) + "="*30)
    if len(len_list)>1:
        None_cols = np.any(None_cols, 0)
        good_cols = np.where(np.logical_not(None_cols))[0]
        print(("{:<10}:"+ " {:<10}"*len(good_cols)).format(
            "Checker", *[DISPS[gidx] for gidx in good_cols])+" {:<10}".format(DISPS[-1]))
        min_len = np.min(len_list[1:])
        succ_vec_all = np.all([svec[:min_len] for svec in succ_vec_list[1:]], axis=0)
        idc_succ_all = np.where(succ_vec_all)[0]

        hard_vec_list = []
        for cname in CNAME_LIST:
            if cname in res_dict:
                vv = np.array(res_dict[cname])[:, good_cols]
                if len(vv)>0:
                    succ_vec = vv[:,-1]
                    fail_vec = np.logical_not(succ_vec)
                    hard_vec_list.append(vv[:min_len, 0] > np.mean(vv[:min_len, 0])*1.0)
                    if cname == "None":
                        idc_succ_now = np.where(succ_vec)[0]
                        idc_fail_none = np.where(fail_vec)[0]
                    else:
                        idc_succ_now = idc_succ_all
                    print(
                        "{:<10}:".format(cname) 
                        + (" {:<10}"*len(good_cols)+" {:<10}").format(
                            *(list(np.round(np.mean(vv[idc_succ_now, :-1], axis=0).astype(np.float), 2)) \
                              +[np.sum(fail_vec), len(vv)])))

        print("="*30 + " {:<10} ".format(ekey+"-hard") + "="*30)
        print(("{:<10}:"+ " {:<10}"*len(good_cols)).format(
            "Checker", *[DISPS[gidx] for gidx in good_cols])+" {:<10}".format(DISPS[-1]))
        hard_vec_all = np.sum(hard_vec_list, axis=0)>=1
        idc_hard_all = np.where(hard_vec_all)[0]
        idc_hard = np.where(np.logical_and(succ_vec_all, hard_vec_all))[0]
        for cname in CNAME_LIST[1:]:
            if cname in res_dict:
                vv = np.array(res_dict[cname])[:, good_cols]
                if len(vv)>0:
                    succ_vec = vv[:,-1]
                    fail_vec = np.logical_not(succ_vec)[idc_hard_all]
                    idc_succ_now = idc_hard
                    print(
                        "{:<10}:".format(cname) 
                        + (" {:<10}"*len(good_cols)+" {:<10}").format(
                            *(list(np.round(np.mean(vv[idc_succ_now, :-1], axis=0).astype(np.float), 2)) \
                              +[np.sum(fail_vec), len(idc_hard_all)])))

============================== obj_1      ==============================
Checker   : Time       Actions    MP_tot     MPs        MP fails   IK_tot     IKs        IK fails   Failures   Total     
None      : 14.65      6.7        3.94       3.94       0.04       54.79      54.79      40.77      3          50        
Tool      : 10.49      6.72       4.3        4.3        0.11       56.54      21.46      11.39      1          50        
ToolReach : 7.93       5.87       4.26       4.26       0.15       78.96      16.91      5.04       1          49        
Full      : 7.09       6.48       3.91       3.91       0.07       60.04      16.59      5.11       1          49        
============================== obj_1-hard ==============================
Checker   : Time       Actions    MP_tot     MPs        MP fails   IK_tot     IKs        IK fails   Failures   Total     
Tool      : 13.75      7.53       4.8        4.8        0.1        71.0       27.67      14.53      1          33        


### Result bak: no latticed before IK
```
============================== obj_1      ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
None      : 9.32       7.0        4.2        0.1        43.4       33.4       0          10        
Tool      : 6.48       6.2        4.8        0.1        20.9       9.5        0          10        
ToolReach : 3.92       5.4        3.5        0.1        8.3        1.7        0          10        
Full      : 3.98       6.2        4.1        0.2        16.6       3.7        0          10        
============================== obj_1-hard ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
Tool      : nan        nan        nan        nan        nan        nan        0          0         
ToolReach : nan        nan        nan        nan        nan        nan        0          0         
Full      : nan        nan        nan        nan        nan        nan        0          0         
============================== obj_3      ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
None      : 29.75      7.5        4.38       0.12       93.31      70.94      0          16        
Tool      : 4.35       6.07       3.53       0.0        12.33      5.8        0          16        
ToolReach : 13.8       6.33       3.67       0.0        25.8       9.67       0          15        
Full      : 5.84       6.87       4.4        0.2        22.2       7.8        0          15        
============================== obj_3-hard ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
Tool      : 2.13       5.0        3.0        0.0        9.0        1.0        0          1         
ToolReach : 154.05     9.0        5.0        0.0        62.0       30.0       0          1         
Full      : 9.39       9.0        5.0        0.0        48.0       22.0       0          1    
```

## Res - panda
```

```

## Res - indy7
```
============================== obj_1           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 7.2             5.8             3.4             0.0             0               10             
Tool           : 7.83            6.5             4.5             0.3             0               10             
ToolReach      : 3.55            5.8             4.2             0.3             0               10             
Full           : 1.94            5.4             3.2             0.0             0               10             
 
============================== obj_3           ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 23.27           8.11            4.56            0.0             1               10             
Tool           : 11.77           5.5             6.12            0.12            2               10             
ToolReach      : 5.02            5.57            9.0             0.0             3               10             
Full           : 6.66            5.44            3.56            0.0             1               10             
 
============================== obj_3_pole      ==============================
Checker        : Planning time   Motion count    MP trials       Failed MP       Failures        Total          
None           : 48.29           9.0             5.0             0.0             4               10             
Tool           : 15.81           6.0             11.75           1.12            2               10             
ToolReach      : 31.28           7.0             14.67           1.17            4               10             
Full           : 15.32           5.0             5.0             0.0             5               10            
```

## Res - indy7 - new
```
============================== obj_1      ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
None      : 11.62      6.53       3.87       0.04       38.17      30.3       3          50        
Tool      : 7.75       5.9        4.75       0.33       24.0       12.96      2          50        
ToolReach : 3.13       5.17       3.19       0.06       6.12       1.67       0          50        
Full      : 2.29       5.58       3.54       0.1        8.1        2.06       0          50        
============================== obj_1-hard ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
Tool      : 14.99      6.05       6.68       0.74       45.53      24.42      0          19        
ToolReach : 5.81       5.42       3.47       0.16       6.84       1.63       0          19        
Full      : 3.41       6.05       4.0        0.21       11.05      2.16       0          19        
============================== obj_3      ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
None      : 19.86      7.19       4.34       0.08       61.74      50.17      8          61        
Tool      : 10.1       6.84       4.2        0.1        30.48      16.24      8          61        
ToolReach : 5.66       6.2        3.78       0.08       25.04      7.42       5          61        
Full      : 6.07       6.08       4.02       0.1        22.0       6.5        6          61        
============================== obj_3-hard ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
Tool      : 26.22      9.0        6.0        0.17       83.83      50.33      0          6         
ToolReach : 15.82      7.0        4.0        0.0        65.33      18.17      0          6         
Full      : 13.71      6.33       6.0        0.17       59.83      17.67      0          6         
============================== obj_3_pole ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
None      : 32.5       7.15       5.55       0.61       114.12     100.67     17         50        
Tool      : 19.07      5.84       5.19       0.51       44.76      25.49      10         50        
ToolReach : 14.79      5.97       5.97       0.65       51.27      20.22      5          50        
Full      : 10.58      6.03       6.81       0.76       38.32      17.24      8          50        
============================== obj_3_pole-hard ==============================
Checker   : Plan time  Actions    MPs        MP fails   IKs        IK fails   Failures   Total     
Tool      : 34.47      7.4        4.2        0.0        61.6       29.2       0          5         
ToolReach : 47.08      6.6        3.8        0.0        199.4      74.8       0          5         
Full      : 22.43      7.2        12.2       1.8        83.2       37.2       0          5        
```